### Company X Bias Analysis

#### Imports

In [77]:
#Standard
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#AIF360 Dataset Classes 
from aif360.datasets import StandardDataset
from aif360.datasets import BinaryLabelDataset

#AIF360 Metrics Classes  
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from aif360.metrics import ClassificationMetric

#AIF360 Algorithms 
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.inprocessing import AdversarialDebiasing
# AI360 Explainers
from aif360.explainers import MetricTextExplainer

#Sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.model_selection import GridSearchCV, train_test_split

#Tensorflow -- install version before 2.0
import tensorflow as tf 

#For Markdown
from IPython.display import Markdown, display

#Helper Functions
from src.classifier_functions import *

#### Load Data


In [11]:
data = pd.read_csv('../company_x.csv', index_col='employee_id')

In [12]:
data.head()


,signing_bonus,salary,degree_level,sex,yrs_experience,dept,is_manager,direct_reports,boss_id
employee_id,,,,,,,,,
138719,0,273000.0,Master,M,2,engineering,False,0.0,43602.0
3192,0,301000.0,Bachelor,F,1,sales,False,0.0,87847.0
114657,0,261000.0,Master,F,2,sales,False,0.0,180854.0
29039,0,86000.0,High_School,F,4,HR,False,0.0,88370.0
118607,0,126000.0,Bachelor,F,3,sales,False,0.0,23565.0


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 138719 to 72227
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   signing_bonus   10000 non-null  int64  
 1   salary          10000 non-null  float64
 2   degree_level    10000 non-null  object 
 3   sex             10000 non-null  object 
 4   yrs_experience  10000 non-null  int64  
 5   dept            10000 non-null  object 
 6   is_manager      10000 non-null  bool   
 7   direct_reports  10000 non-null  float64
 8   boss_id         9999 non-null   float64
dtypes: bool(1), float64(3), int64(2), object(3)
memory usage: 712.9+ KB


In [24]:
## Create Label 
data_with_label = data.copy()
data_with_label['salary'] = data_with_label['salary'].transform(lambda x: x > 150000).astype(int)
data_with_label['sex'] = data_with_label['sex'].transform(lambda x: x == 'M').astype(int)
data_with_label.head()

,signing_bonus,salary,degree_level,sex,yrs_experience,dept,is_manager,direct_reports,boss_id
employee_id,,,,,,,,,
138719,0,1,Master,1,2,engineering,False,0.0,43602.0
3192,0,1,Bachelor,0,1,sales,False,0.0,87847.0
114657,0,1,Master,0,2,sales,False,0.0,180854.0
29039,0,0,High_School,0,4,HR,False,0.0,88370.0
118607,0,0,Bachelor,0,3,sales,False,0.0,23565.0


In [26]:
### Create StandardDataset

std_data = StandardDataset(df=data_with_label,   
                         label_name='salary',
                         favorable_classes =[1],
                        protected_attribute_names=['sex'], 
                         privileged_classes=[[1]],
                        categorical_features=['degree_level', 'dept'], 
                          features_to_drop=['boss_id', 'is_manager'])



In [27]:
df_data = std_data.convert_to_dataframe()
binary_data = BinaryLabelDataset(favorable_label=1, unfavorable_label=0, df=df_data[0], label_names=['salary'],
              protected_attribute_names=['sex'])

privileged_groups= [{'sex':1}]
unprivileged_groups= [{'sex': 0}]

In [28]:
## Splitting the Data
data_train, data_vt = binary_data.split([0.7], shuffle=True)

data_val, data_test = binary_data.split([0.5], shuffle=True)

In [34]:
### First Metrics
metric_train = BinaryLabelDatasetMetric(data_train, 
                            unprivileged_groups=unprivileged_groups, 
                            privileged_groups=privileged_groups)
#Explainer 
ex_metric_train = MetricTextExplainer(metric_train)

print(ex_metric_train.mean_difference())
print('\n')
print(ex_metric_train.disparate_impact())

Mean difference (mean label value on privileged instances - mean label value on unprivileged instances): -0.1490216737295076


Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.7756241576968832


### Random Forest Classifier 

In [36]:
#Creating X,y
X_train = data_train.features
y_train = data_train.labels.ravel()
X_test = data_val.features
y_test = data_val.labels.ravel()

In [51]:
# Creating Random Forest Classifier
rfc = RandomForestClassifier(n_estimators=1000, max_depth=5)
rfc.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [52]:
#Creating Predicted Dataframe
y_pred = rfc.predict(X_test)
data_val_pred = data_val.copy()
data_val_pred.labels = y_pred

In [54]:
rfc.score(X_test, y_test)

0.7738

In [55]:
#Getting the Metrics 
metric_preds = BinaryLabelDatasetMetric(data_val_pred, 
                            unprivileged_groups=unprivileged_groups, 
                            privileged_groups=privileged_groups)

ex_metric_preds = MetricTextExplainer(metric_preds)

print(ex_metric_preds.mean_difference())
print('\n')
print(ex_metric_preds.disparate_impact())


model_metric = ClassificationMetric(data_val, data_val_pred, 
                            unprivileged_groups=unprivileged_groups, 
                            privileged_groups=privileged_groups)

ex_model_metric= MetricTextExplainer(model_metric)
print('\n')
print(ex_model_metric.recall())
print('\n')
print(ex_model_metric.precision())
print('\n')
print(ex_model_metric.average_odds_difference())
print('\n')
print(ex_model_metric.equal_opportunity_difference())
print('\n')
print(ex_model_metric.theil_index())

Mean difference (mean label value on privileged instances - mean label value on unprivileged instances): -0.21191370063682125


Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.7669382359140682


True positive rate (TPR, recall, sensitivity = TP / (TP + FN)): 0.9977079240340537


Positive predictive value (PPV, precision = TP / (TP + FP)): 0.730520258930712


Average odds difference (average of TPR difference and FPR difference, 0 = equality of odds): -0.1703981167343366


True positive rate difference (true positive rate on unprivileged instances - true positive rate on privileged instances): -0.0014946461670859446


Theil index (generalized entropy index with alpha = 1): 0.053098004316676814


## Debiasing through Preprocessing 

#### Reweighing

In [60]:
RW = Reweighing(unprivileged_groups=unprivileged_groups, 
               privileged_groups=privileged_groups)

#Splitting the Data
data_rw_train = RW.fit_transform(data_train)

data_rw_val = RW.fit_transform(data_val)

array([0.91393568, 0.7877408 , 1.20728437, ..., 0.91393568, 0.91393568,
       0.7877408 ])

In [63]:
### Reweigh Metrics
metric_rw_train = BinaryLabelDatasetMetric(data_rw_train, 
                            unprivileged_groups=unprivileged_groups, 
                            privileged_groups=privileged_groups)
#Explainer 
ex_metric_rw_train = MetricTextExplainer(metric_rw_train)

print(ex_metric_rw_train.mean_difference())
print('\n')
print(ex_metric_rw_train.disparate_impact())

Mean difference (mean label value on privileged instances - mean label value on unprivileged instances): -2.220446049250313e-16


Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.9999999999999997


### Random Forest Classifier with Reweighing 

In [65]:
#Creating X,y
X_rw_train = data_rw_train.features
y_rw_train = data_rw_train.labels.ravel()
X_rw_test = data_rw_val.features
y_rw_test = data_rw_val.labels.ravel()

In [87]:
# Creating Random Forest Classifier
rfc_rw = RandomForestClassifier(n_estimators=1000, max_depth=5)
rfc_rw.fit(X_rw_train, y_rw_train, sample_weight=data_transf_train.instance_weights)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [88]:
#Creating Predicted Dataframe
y_rw_pred = rfc_rw.predict(X_test)
data_val_rw_pred = data_rw_val.copy()
data_val_rw_pred.labels = y_rw_pred

In [150]:
#Getting the Metrics 
display(Markdown("#### Model - with reweighing - dataset metrics"))


metric_rw_preds = BinaryLabelDatasetMetric(data_val_rw_pred, 
                            unprivileged_groups=unprivileged_groups, 
                            privileged_groups=privileged_groups)

ex_metric_rw_preds = MetricTextExplainer(metric_rw_preds)

print(ex_metric_rw_preds.mean_difference())
print('\n')
print(ex_metric_rw_preds.disparate_impact())

display(Markdown("#### Model - with reweighing - classification metrics"))
model_rw_metric = ClassificationMetric(data_rw_val, data_val_rw_pred, 
                            unprivileged_groups=unprivileged_groups, 
                            privileged_groups=privileged_groups)

ex_model_rw_metric= MetricTextExplainer(model_rw_metric)

print(ex_model_rw_metric.recall())
print(ex_model_rw_metric.precision())
print(ex_model_rw_metric.average_odds_difference())
print(ex_model_rw_metric.equal_opportunity_difference())
print(ex_model_rw_metric.theil_index())
print(ex_model_rw_metric.accuracy(privileged=True))
print(ex_model_rw_metric.accuracy(privileged=False))

#### Model - with reweighing - dataset metrics

Mean difference (mean label value on privileged instances - mean label value on unprivileged instances): -0.13228741122806253


Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.8520019469702573


#### Model - with reweighing - classification metrics

True positive rate (TPR, recall, sensitivity = TP / (TP + FN)): 0.9980123407089898
Positive predictive value (PPV, precision = TP / (TP + FP)): 0.7197090367884792
Average odds difference (average of TPR difference and FPR difference, 0 = equality of odds): -0.1698400810200509
True positive rate difference (true positive rate on unprivileged instances - true positive rate on privileged instances): -0.0003785747385145477
Theil index (generalized entropy index with alpha = 1): 0.052892902386780814
Classification accuracy on privileged instances: 0.714690062815364
Classification accuracy on unprivileged instances: 0.846515007142857


In [79]:
scorecard(y_rw_test, y_rw_pred)

The Accuracy score is 0.774.

The Precision score is 0.731.

The Recall score is 0.998.

      Confusion Matrix


,Actual True,Actual False
Predicted True,3047,1124
Predicted False,7,822


### Inprocessing with Adversarial Debiaising



In [153]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

In [154]:
##Creating Tensorflow Session - must not use Tensorflow 2.0 
sess = tf.Session()
db_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                            num_epochs  = 100,
                            batch_size = 100,
                            adversary_loss_weight = .1,     
                            debias=True,
                          sess=sess)

In [155]:
db_model.fit(data_train) 

epoch 0; iter: 0; batch classifier loss: 0.691504; batch adversarial loss: 0.749611
epoch 1; iter: 0; batch classifier loss: 0.587889; batch adversarial loss: 0.759727
epoch 2; iter: 0; batch classifier loss: 0.464046; batch adversarial loss: 0.709005
epoch 3; iter: 0; batch classifier loss: 0.505298; batch adversarial loss: 0.703604
epoch 4; iter: 0; batch classifier loss: 0.473328; batch adversarial loss: 0.694708
epoch 5; iter: 0; batch classifier loss: 0.423008; batch adversarial loss: 0.684895
epoch 6; iter: 0; batch classifier loss: 0.453056; batch adversarial loss: 0.684875
epoch 7; iter: 0; batch classifier loss: 0.487496; batch adversarial loss: 0.683074
epoch 8; iter: 0; batch classifier loss: 0.392686; batch adversarial loss: 0.613378
epoch 9; iter: 0; batch classifier loss: 0.413701; batch adversarial loss: 0.650609
epoch 10; iter: 0; batch classifier loss: 0.460129; batch adversarial loss: 0.708791
epoch 11; iter: 0; batch classifier loss: 0.533745; batch adversarial loss:

In [156]:
dataset_debiasing_train = db_model.predict(data_train)
dataset_debiasing_test = db_model.predict(data_val)

In [157]:

# Metrics for the dataset from model with debiasing
display(Markdown("#### Model - with debiasing - dataset metrics"))
metric_dataset_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_train.mean_difference())

metric_dataset_debiasing_test = BinaryLabelDatasetMetric(dataset_debiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_test.mean_difference())



display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(data_val, 
                                                 dataset_debiasing_test,
                                                    unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test.theil_index())
                                                        
                                                        

#### Model - with debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.184829
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.198142


#### Model - with debiasing - classification metrics

Test set: Classification accuracy = 0.775400
Test set: Balanced classification accuracy = 0.713417
Test set: Disparate impact = 0.779151
Test set: Equal opportunity difference = 0.008152
Test set: Average odds difference = -0.154266
Test set: Theil_index = 0.055493


In [ ]:
### Postprocessing with ? 